# Tracking the NASA Satellite
For this assignment, we will be working on streaming data which is satellite location coordinates being provided by NASA.

The data is in the following format:
{"timestamp": 1667492679, "iss_position": {"longitude": "-56.8155", "latitude": "-42.5979"}, "message": "success"}

Data can be accessed from http://api.open-notify.org/iss-now.json

The link can be thought of as the producer as it will do producer job.

Write the consumer to connect and fetch data. In the consumer, you have to write the code which takes the data from producer and uses the location coordinates to plot the satellite location on the world map.

The streaming interval should be 5 seconds. The producer should run for an hour. The graph will show the satellite tracking for 1 hr.

P.S.: Keeping in mind that satellite travels very fast, the graph will have more than half of map covered with satellite track

# Solution
Here we use Folium library to create an interactive map tracking the International Space Station (ISS) as it moves in real-time. 

The ISS's current location coordinates is fetched and streamed from the NASA API for one hour with five second intervels, updates the map, and displays the path of the ISS as it orbits the Earth. 

Additionally, the script marks the start and end coordinates of the ISS's path along with timestamps.

In [37]:
!pip install folium

In [26]:
import time
import requests
import folium

# Defining API endpoint and ISS tracking parameters
api_url = "http://api.open-notify.org/iss-now.json"
duration = 3600  # 1 hour in seconds
interval = 5  # 5 seconds
coordinates = []
end_time = time.time() + duration  # Calculate the end time


# Initializing the map
m = folium.Map(location=[0, 0], zoom_start=2)

# Defining function to fetch and streaming the data from the API and update the map

Using requests package a get request is called to fetch the coordinates of ISS 

In [27]:
def fetch_and_update_map():
    global coordinates
    response = requests.get(api_url)
    data = response.json()
    if "iss_position" in data:
        longitude = float(data["iss_position"]["longitude"])
        latitude = float(data["iss_position"]["latitude"])
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(data["timestamp"]))

        # If coordinates are present, add them to the map and update the path
        if coordinates:
            # Create a polyline between the current and previous coordinates
            folium.PolyLine(locations=[coordinates[-1], [latitude, longitude]], color="blue").add_to(m)

        # Add a marker for the current coordinate with a timestamp
        folium.Marker(location=[latitude, longitude], icon=folium.Icon(color='green'),
                      popup=f'Current ({latitude}, {longitude}) - {timestamp}').add_to(m)

        coordinates.append([latitude, longitude])


During the duration of one hour the fetch and update map method is called iteratively the update and plot the coordinates of ISS on world map

In [28]:
# Run the tracking for the specified duration
while time.time() < end_time:
    fetch_and_update_map()
    time.sleep(interval)

# Add the end marker
if coordinates:
    latitude, longitude = coordinates[-1]
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
    folium.Marker(location=[latitude, longitude], icon=folium.Icon(color='red'),
                  popup=f'End ({latitude}, {longitude}) - {timestamp}').add_to(m)

# Save the final map
m.save("iss_tracking.html") # the output is shown in iss_tracking.html file